In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.misc as sm

(5412, 7216)


In [9]:
# sample from the model
new_x_gen = []
for i in range(args.nr_gpu):
    with tf.device('/gpu:%d' % i):
        gen_par = model(xs[i], h_sample[i], ema=ema, dropout_p=0, **model_opt)
        new_x_gen.append(nn.sample_from_discretized_mix_logistic(gen_par, args.nr_logistic_mix))
def sample_from_model(sess):
    x_gen = [samples[i] for i in range(args.nr_gpu)]
    for yi in range(obs_shape[0]):
        for xi in range(obs_shape[1]):
            new_x_gen_np = sess.run(new_x_gen, {xs[i]: x_gen[i] for i in range(args.nr_gpu)})
            for i in range(args.nr_gpu):
                x_gen[i][:,yi,xi,:] = new_x_gen_np[i][:,yi,xi,:]*(1-masks[:,yi,xi,:])+x_gen[i][:,yi,xi,:]*masks[:,yi,xi,:]
    return np.concatenate(x_gen, axis=0)



# train for one epoch
train_losses = []
for d in tqdm(train_data):
    feed_dict = make_feed_dict(d)
    # forward/backward/update model on each gpu
    lr *= args.lr_decay
    feed_dict.update({ tf_lr: lr })
    l,_ = sess.run([bits_per_dim, optimizer], feed_dict)
    train_losses.append(l)
train_loss_gen = np.mean(train_losses)

# compute likelihood over test data
test_losses = []
for d in test_data:
    feed_dict = make_feed_dict(d)
    l = sess.run(bits_per_dim_test, feed_dict)
    test_losses.append(l)
test_loss_gen = np.mean(test_losses)
test_bpd.append(test_loss_gen)

# log progress to console
print("Iteration %d, time = %ds, train bits_per_dim = %.4f, test bits_per_dim = %.4f" % (epoch, time.time()-begin, train_loss_gen, test_loss_gen))
sys.stdout.flush()

if epoch % args.save_interval == 0:

    
    for d in test_data:
        feed_dict = make_feed_dict(d)
        l = sess.run(bits_per_dim_test, feed_dict)
        test_losses.append(l)
    
    # generate samples from the model
    sample_x = sample_from_model(sess)
    img_tile = plotting.img_tile(sample_x[:int(np.floor(np.sqrt(args.batch_size*args.nr_gpu))**2)], aspect_ratio=1.0, border_color=1.0, stretch=True)
    img = plotting.plot_img(img_tile, title=args.data_set + ' samples')
    plotting.plt.savefig(os.path.join(args.save_dir,'%s_sample%d.png' % (args.data_set, epoch)))
    plotting.plt.close('all')



(array([  0,   0,   0, ..., 383, 383, 383]), array([359, 360, 361, ..., 381, 382, 383]))
